#### Introduction
- このコードは"OCR model for reading Captchas"を可変長のテキストを持つIAMデータセットに拡張する方法について取り上げる。
- データセットの各サンプルは手書きテキストの画像で、対応するターゲットは画像に存在する文字列である。

#### Data
- データセットがどのように構成されているかプレビューする。
- #で始まる行は単なるメタデータである。

In [1]:
!head -20 data/IAM_Words/words.txt

#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
a01-000u-00-00 ok 154 408 768 27 51 AT A
a01-000u-00-01 ok 154 507 766 213 48 NN MOVE


#### Setup

In [2]:
from keras.layers import StringLookup
import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

#### Dataset splitting

In [3]:
base_path = "data/IAM_Words/"
words_list = []

words = open(f"{base_path}/words.txt", "r").readlines()
for line in words:
    if line[0] == "#":
        continue
    if line.split(" ")[1] != "err": # エラーを除外
        words_list.append(line)

print(len(words_list))
np.random.shuffle(words_list)

96456


In [4]:
# データセットを train:90, validation:5, test:5 の割合で分割
split_idx = int(0.9 * len(words_list))
train_samples = words_list[:split_idx]
test_samples = words_list[split_idx:]

val_split_idx  = int(0.5 * len(test_samples))
validation_samples = test_samples[:val_split_idx]
test_samples = test_samples[val_split_idx:]

assert len(words_list) == len(train_samples) + len(validation_samples) + len(test_samples)

print(f"Total training samples: {len(train_samples)}")
print(f"Toral validation samples: {len(validation_samples)}")
print(f"Total test samples: {len(test_samples)}")

Total training samples: 86810
Toral validation samples: 4823
Total test samples: 4823


#### Data input pipeline

In [5]:
# 画像パスの準備
base_image_path = os.path.join(base_path)

def get_image_paths_and_labels(samples):
    paths = []
    corrected_samples = []
    for (i, file_line) in enumerate(samples):
        line_split = file_line.strip()
        line_split = line_split.split(" ")

        # 各行が分割されると、対応する画像は次のような書式になる: 
        # part1/part1-part2/part1-part2-part3.png
        image_name = line_split[0]
        partI = image_name.split("-")[0]
        partII = image_name.split("-")[1]
        img_path = os.path.join(
            base_image_path, partI, partI + "-" + partII, image_name + ".png"
        )
        if os.path.getsize(img_path):
            paths.append(img_path)
            corrected_samples.append(file_line.split("\n")[0])

    return paths, corrected_samples

train_img_paths, train_labels = get_image_paths_and_labels(train_samples)
validation_img_paths, validation_labels = get_image_paths_and_labels(validation_samples)
test_img_paths, test_labels = get_image_paths_and_labels(test_samples)

In [6]:
# 訓練データから最大長および語彙のサイズを求める
train_labels_cleaned = []
characters = set()
max_len = 0

for label in train_labels:
    label = label.split(" ")[-1].strip()
    for char in label:
        characters.add(char)

    max_len = max(max_len, len(label))
    train_labels_cleaned.append(label)

characters = sorted(list(characters))

print(f"Maximum length: {max_len}")
print(f"Vocab size: {len(characters)}")

# ラベルの確認
train_labels_cleaned[:10]

Maximum length: 21
Vocab size: 78


['Hahnemann',
 'qualified',
 'held',
 'brought',
 '.',
 'the',
 'in',
 'unruffled',
 '.',
 'I']

In [7]:
def clean_label(labels):
    cleaned_labels = []
    for label in labels:
        label = label.split(" ")[-1].strip()
        cleaned_labels.append(label)
    return cleaned_labels

validation_labels_cleaned = clean_label(validation_labels)
test_labels_cleaned = clean_label(test_labels)

#### Buiding the character vocabulary

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

# 文字を整数にマッピング
char_to_num = StringLookup(vocabulary=list(characters), mask_token=None)

# 整数から元の文字にマッピング
num_to_char = StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

#### Resizing images without distortion
- 正方形の画像ではなく、多くのOCRモデルでは長方形の画像を扱う
- アスペクトを意識しない正方形画像のリサイズでは、大きな歪みは生じないが、長方形の画像の場合はそうではない
- 画像を均一にリサイズすることは、ミニバッチの要件である。そこで、以下の基準を満たすようにリサイズを行う必要がある。
    - アスペクト比が保持される
    - 画像の内容が影響を受けない

In [9]:
def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h,w), preserve_aspect_ratio=True)

    # 必要なパッドの量を確認する
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # 両側に同じ量のパッドを入れる場合のみ必要
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2
    
    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0]
        ]
    )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image